In [31]:
import pandas as pd
import re

#Read in the latest job scrape
df = pd.read_json('output/glassdoor-jobs_2021-06-10_19:10:47.json')

#Remove the column name prefixes (artefacts from scraping process)
def strip_pref(x):
    return x[len('data_jobView_header_'):] if 'data_jobView_header_' in x else x[len('data_jobView_'):]
df.columns = [strip_pref(p) for p in df.columns]

#Just pull out the most basic columns for now, we're gonna focus on the job_description and do some NLP
#The other values we scraped are kinda useless
column_names = [
    'ageInDays',
    'employer_name',
    'employer_size',
    'expired',
    'locationName',
    'normalizedJobTitle',
    'job_description',
    'job_listingId'
]
df = df[column_names]

#And normalise the column names once more into snake_case
pattern = re.compile(r'(?<!^)(?=[A-Z])')
df.columns = [pattern.sub('_', name).lower() for name in df.columns]

#Use the job_listing_id as an index (TODO - will have to watch out for this if we add a more sources)
df = df.set_index('job_listing_id')
df.head()

,age_in_days,employer_name,employer_size,expired,location_name,normalized_job_title,job_description
job_listing_id,,,,,,,
4086170095,0,"HubSpot, Inc.",1.001 bis 5.000 Mitarbeiter,False,Berlin,Technical Support,"As a Customer Support Specialist, you will be ..."
4086170160,0,"HubSpot, Inc.",1.001 bis 5.000 Mitarbeiter,False,Berlin,Account Executive,"As a Remote Account Executive at HubSpot, you ..."
4109534176,0,Lebenswelt gGmbH,Unbekannt,False,Berlin,Arbeiter (M/W/D),Zur Verst&auml;rkung unserer Teams in den Ambu...
4109016075,0,Deutsche See Gmbh,1.001 bis 5.000 Mitarbeiter,False,Berlin,Vertriebsmanager (M/W/D),<div><p>Deutsche See GmbH ist nationaler Markt...
4086170059,0,"HubSpot, Inc.",1.001 bis 5.000 Mitarbeiter,False,Berlin,Account Executive,<b>Start :</b> Q1/ 2021<br/><b>Location :</b> ...
